In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import RootMeanSquaredError
import seaborn as sns


In [2]:
#Importing data
data = pd.read_excel("C:/Users/orkun/OneDrive/Masaüstü/Ens 492/Code/Last versions/Simdatawithoutspindlevar.xlsx")
data = data.sample(n=35000, random_state=42)# Sampling random 35000 simulations for increasing the speed of ML training process 

data.head()

#splitting features and targets (ignore spindle speed due to it is same for all the simulations)
X = data.drop(columns=['Spindle_Speed', 'Feed_Rate', 'Axial_Depth', 'Radial_Depth'])
y = data[['Feed_Rate', 'Axial_Depth', 'Radial_Depth']]

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_standardized = scaler_X.fit_transform(X)
y_standardized = scaler_y.fit_transform(y)

#Splitting train and test sets (%80, %20)
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y_standardized, test_size=0.2, random_state=42)

# Reshaping the data for Neural Networks
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [3]:
# Model Definition
def create_model(hidden_units=64, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(hidden_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(hidden_units*2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(hidden_units, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear')) # Çıkış katmanı, tahmin edilecek değer sayısı
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=[RootMeanSquaredError()])
    return model

In [7]:
# Hyperparameters
hyperparameters = {
    'hidden_units': [32, 64],
    'dropout_rate': [0.2, 0.3],
    'learning_rate': [0.001, 0.0001]
}

results = []
best_model = None
best_params = None
best_rmse = float('inf')

num_epochs=30
# Testing different hyperparameter combinations
for hidden_units in hyperparameters['hidden_units']:
    for dropout_rate in hyperparameters['dropout_rate']:
        for learning_rate in hyperparameters['learning_rate']:
            model = create_model(hidden_units=hidden_units, dropout_rate=dropout_rate, learning_rate=learning_rate)
            history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2, verbose=0)
            loss, rmse = model.evaluate(X_test, y_test, verbose=0)
            results.append({
                'hidden_units': hidden_units,
                'dropout_rate': dropout_rate,
                'learning_rate': learning_rate,
                'rmse': rmse
            })

            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_params = {
                    'hidden_units': hidden_units,
                    'dropout_rate': dropout_rate,
                    'learning_rate': learning_rate
                }


results_df = pd.DataFrame(results)



In [8]:
print(f"Best Parameters: {best_params}")

# Making predictions with the best model
y_pred = best_model.predict(X_test)

# Inversely transforming the test data to get the original scale values
y_test_unscaled = scaler_y.inverse_transform(y_test)
y_pred_unscaled = scaler_y.inverse_transform(y_pred)

# RMSE Calculation
rmse_feed_rate = np.sqrt(np.mean((y_test_unscaled[:, 0] - y_pred_unscaled[:, 0])**2))
rmse_axial_depth = np.sqrt(np.mean((y_test_unscaled[:, 1] - y_pred_unscaled[:, 1])**2))
rmse_radial_depth = np.sqrt(np.mean((y_test_unscaled[:, 2] - y_pred_unscaled[:, 2])**2))

print(f"RMSE for Feed Rate: {rmse_feed_rate}")
print(f"RMSE for Axial Depth: {rmse_axial_depth}")
print(f"RMSE for Radial Depth: {rmse_radial_depth}")


Best Parameters: {'hidden_units': 64, 'dropout_rate': 0.2, 'learning_rate': 0.0001}
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
RMSE for Feed Rate: 0.011594382900001693
RMSE for Axial Depth: 0.387812953352265
RMSE for Radial Depth: 0.1979190036574621


In [9]:
#Error margin
error_margin = 0.1
y_pred_lower = y_pred_unscaled * (1 - error_margin)
y_pred_upper = y_pred_unscaled * (1 + error_margin)

# Function to determine given values are valid or not
def is_within_error_margin(y_true, y_pred_lower, y_pred_upper):
    return np.all((y_true >= y_pred_lower) & (y_true <= y_pred_upper), axis=1)

# Doğru sınıflandırma oranı hesaplama
correct_classifications = is_within_error_margin(y_test_unscaled, y_pred_lower, y_pred_upper)
accuracy = np.mean(correct_classifications) * 100

print(f"Correct classification rate(%10): {accuracy:.2f}%")



Correct classification rate(%10): 61.21%
